In [ ]:
!pip install dspy micropip mlflow

In [ ]:
!setup_sitcom

In [ ]:
!curl -fsSL https://deno.land/install.sh | sh

# Deno Install 

`curl -fsSL https://deno.land/install.sh | sh`

Create a file in the library of DSPy

`~/.local/lib/python3.11/site-packages/dspy/primitives/runner.js`

```javascript
import pyodideModule from "npm:pyodide/pyodide.js";
import { readLines } from "https://deno.land/std@0.186.0/io/mod.ts";

const pyodide = await pyodideModule.loadPyodide();

for await (const line of readLines(Deno.stdin)) {
    let input;
    try {
        input = JSON.parse(line);
    } catch (error) {
        console.log(JSON.stringify({ error: "Invalid JSON input: " + error.message }));
        continue;
    }

    if (typeof input !== 'object' || input === null) {
        console.log(JSON.stringify({ error: "Input is not a JSON object" }));
        continue;
    }

    if (input.shutdown) {
        break;
    }

    let output;
    try {
        const result = await pyodide.runPythonAsync(input.code || "");
        output = JSON.stringify({ output: result });
    } catch (error) {
        output = JSON.stringify({ error: error.message.trim().split('\n').pop() || ''});
    }
    console.log(output);
}
```

# Multi-Agents System for General Analysis


In [ ]:
import os
import dspy
import micropip
from dspy.primitives.python_interpreter import PythonInterpreter

# Intenta obtener el valor de la variable de entorno
api_base = os.environ.get("API_BASE")

# Si no se encuentra, lee la URL desde el archivo
filename = os.path.expanduser("~/.base_key.txt")

if not api_base:
    with open(filename, "r") as f:
        api_base = f.read().strip()

#R1 = dspy.LM('ollama_chat/R1_70b_F16',
tulu70 = dspy.LM('ollama_chat/tulu3:70b',
# tulu8 = dspy.LM('ollama_chat/tulu3:8b',
             api_base=api_base,
             api_key='')

# Configuración (asegúrate de que 'tulu' esté definido o usa 'R1' si ese es el LM que deseas configurar)
# dspy.configure(lm=tulu)
dspy.configure(lm=tulu8)

%matplotlib inline

In [ ]:
summarize = dspy.ChainOfThought('document -> summary')
response = summarize(document="We had a productive night on AOS commissioning. We targeted several open clusters at different elevations to look for chromatic effects in wavefront estimation in g-band. Along the way, we completely by-accident ended up near the Orion Nebula and found some really spectacular ghosts; enough to make us think of some new alignment possibilities using spider shadows tomorrow (spooky!). We took wavefront repeatability data and investigated reweighting different degrees of freedom in an attempt to more efficiently squash spherical aberration. That approach turned out to not be successful, but we got some good data - including closed loop repeatability data - to chew on until tomorrow. Finally, we executed the rotator angle lookup test.")
print(response.summary)

In [ ]:
"""
Multi-Agent System for General Analysis
with DSPy and Python Code Editing & Execution.

This project demonstrates a multi-agent architecture that includes:
    - PredictorAgent: Provides an initial answer with chain-of-thought reasoning.
    - ReflectorAgent: Reviews and suggests improvements to the initial answer.
    - DebaterAgent: Combines multiple perspectives to refine the answer.
    - PythonCodeExecutor: Generates Python code from the given instruction.
    - CodeEditorAgent: Edits the generated Python code based on the memory of past executions.

The system is generic and includes persistent memory for code execution history.
It is intended to run in a Jupyter Notebook.
"""

# =============================================================================
# 1. IMPORTS AND INITIAL SETUP
# =============================================================================
import logging
from typing import List, Literal

import numpy as np
import pandas as pd

import dspy  # Ensure DSPy is installed: pip install dspy
from dspy.primitives.python_interpreter import PythonInterpreter

# Configure logging
logging.basicConfig(level=logging.INFO)
LOG = logging.getLogger(__name__)

# =============================================================================
# 2. MEMORY CLASS FOR CODE EXECUTIONS
# =============================================================================
class Memory:
    """A simple memory class to store executed code and their results."""
    def __init__(self):
        self.history = []
    
    def add(self, code: str, result: str):
        self.history.append({"code": code, "result": result})
    
    def get_context(self) -> str:
        context = ""
        for entry in self.history:
            context += f"Code: {entry['code']}\nResult: {entry['result']}\n---\n"
        return context

# =============================================================================
# 3. DEFINE DSPy AGENTS (SIGNATURES)
# =============================================================================

# -----------------------------------------------------------------------------
# 3.1 PredictorAgent: Uses a chain-of-thought to answer questions.
# -----------------------------------------------------------------------------
class PredictorAgent(dspy.Signature):
    question: str = dspy.InputField(desc="User question")
    answer: str = dspy.OutputField(desc="Initial answer with reasoning (CoT)")

predictor_agent = dspy.ChainOfThought(PredictorAgent)

# -----------------------------------------------------------------------------
# 3.2 ReflectorAgent: Reviews and suggests improvements.
# -----------------------------------------------------------------------------
class ReflectorAgent(dspy.Signature):
    question: str = dspy.InputField(desc="Original question")
    predictor_answer: str = dspy.InputField(desc="Answer from Predictor")
    suggestion: str = dspy.OutputField(desc="Constructive criticism or suggestion")

reflector_agent = dspy.Predict(ReflectorAgent)

# -----------------------------------------------------------------------------
# 3.3 DebaterAgent: Debates and refines the answer.
# -----------------------------------------------------------------------------
class DebaterAgent(dspy.Signature):
    question: str = dspy.InputField(desc="Original question")
    previous_solutions: str = dspy.InputField(desc="Combined previous answers and critiques")
    debated_answer: str = dspy.OutputField(desc="Final refined answer after debate")

debater_agent = dspy.Predict(DebaterAgent)

# -----------------------------------------------------------------------------
# 3.4 PythonCodeExecutor: Generates Python code from an instruction.
# -----------------------------------------------------------------------------
class PythonCodeExecutor(dspy.Signature):
    code_instruction: str = dspy.InputField(desc="Python code instruction to generate code")
    code_result: str = dspy.OutputField(desc="Generated Python code")

python_executor = dspy.Predict(PythonCodeExecutor)
python_interpreter = PythonInterpreter()

# -----------------------------------------------------------------------------
# 3.5 CodeEditorAgent: Edits the generated Python code based on memory.
# -----------------------------------------------------------------------------
class CodeEditorAgent(dspy.Signature):
    original_code: str = dspy.InputField(desc="Original Python code to be edited")
    memory_context: str = dspy.InputField(desc="Memory context from previous code executions")
    edited_code: str = dspy.OutputField(desc="Edited Python code after considering memory")

code_editor_agent = dspy.Predict(CodeEditorAgent)

# =============================================================================
# 4. DEFINE THE MULTI-AGENT SYSTEM (TOPOLOGY)
# =============================================================================
class MultiAgentSystem(dspy.Module):
    """
    A multi-agent system that integrates:
      - Predictor (with chain-of-thought)
      - Reflector
      - Debater
      - Python code generation, editing, and execution (via DSPy's PythonInterpreter)
      
    The system maintains persistent memory of past code executions.
    """
    def __init__(self):
        super().__init__()
        self.predictor = predictor_agent
        self.reflector = reflector_agent
        self.debater = debater_agent
        self.python_executor = python_executor
        self.code_editor = code_editor_agent
        self.memory = Memory()
    
    def forward(self, question: str) -> str:
        # 1. Predictor generates an initial answer.
        pred_result = self.predictor(question=question).answer
        LOG.info(f"[Predictor]: {pred_result}")

        # 2. Reflector reviews the answer.
        reflection = self.reflector(question=question, predictor_answer=pred_result).suggestion
        LOG.info(f"[Reflector]: {reflection}")

        # 3. Debater refines the answer.
        debate_input = f"Initial Answer: {pred_result}\nReflection: {reflection}"
        debated_result = self.debater(question=question, previous_solutions=debate_input)
        debated = (debated_result.debated_answer 
                   if hasattr(debated_result, 'debated_answer') 
                   else "No debated answer returned")
        LOG.info(f"[Debater]: {debated}")

        final_answer = debated  # The base final answer.
        output = f"Final Answer: {final_answer}"

        lower_q = question.lower()
        # 4. Check if the question requires Python code execution.
        if "execute python" in lower_q or "run code" in lower_q or "python code" in lower_q:
            # Generate Python code from the instruction.
            gen_code = self.python_executor(code_instruction=question).code_result
            LOG.info(f"[PythonCodeExecutor] Generated Code: {gen_code}")

            # Edit the generated code using the CodeEditorAgent, providing memory context.
            memory_context = self.memory.get_context()
            edited_code = self.code_editor(original_code=gen_code, memory_context=memory_context).edited_code
            LOG.info(f"[CodeEditorAgent] Edited Code: {edited_code}")

            # Execute the edited code using PythonInterpreter.
            try:
                exec_result = python_interpreter(edited_code)
                if exec_result is None:
                    exec_result = "No output"
                else:
                    exec_result = str(exec_result)
            except Exception as ex:
                exec_result = f"Error executing code: {ex}"
            LOG.info(f"[PythonInterpreter] Execution Result: {exec_result}")

            # Add the executed code and its result to memory.
            self.memory.add(edited_code, exec_result)

            output += f"\nPython Code Execution Result: {exec_result}"

        return output

# =============================================================================
# 5. EXAMPLES OF USAGE
# =============================================================================
if __name__ == "__main__":
    system = MultiAgentSystem()
    
    # Example 1: A general query without code execution.
    question1 = "What is the meaning of life?"
    print("=== Example 1: General Query ===")
    print("Question:", question1)
    print("Response:", system(question=question1))
    
    # Example 2: A query requiring Python code execution.
    question2 = "Execute Python code to print the sum of numbers from 1 to 10."
    print("\n=== Example 2: Python Code Execution ===")
    print("Question:", question2)
    print("Response:", system(question=question2))
    
    # Example 3: Another code execution to show memory retention.
    question3 = "Run code to calculate the factorial of 5."
    print("\n=== Example 3: Python Code Execution with Memory ===")
    print("Question:", question3)
    print("Response:", system(question=question3))
    
    # Shutdown the PythonInterpreter when done.
    python_interpreter.shutdown()

In [ ]:
# --- New Cell: Iterate over Previous Executions and Use Memory Context for Code Editing ---

# Assume you already have an instance of MultiAgentSystem called "system"
# that has executed some code previously and stored it in its memory.

# 1. Display the previous code execution history.
print("=== Previous Code Execution History ===")
for idx, entry in enumerate(system.memory.history, start=1):
    print(f"Entry {idx}:")
    print("Code:")
    print(entry["code"])
    print("Result:")
    print(entry["result"])
    print("-" * 40)

# 2. Retrieve the aggregated memory context.
memory_context = system.memory.get_context()
print("\n=== Memory Context ===")
print(memory_context)

# 3. Define a new Python code snippet that needs editing.
#    (This snippet can be any new instruction you wish to modify based on past executions.)
new_code_snippet = """
def greet(name):
    return f"Hello, {name}!"

print(greet("World"))
"""

print("\n=== Original New Code Snippet ===")
print(new_code_snippet)

# 4. Use the CodeEditorAgent to edit the new code snippet based on the memory context.
edited_code = system.code_editor(original_code=new_code_snippet, memory_context=memory_context).edited_code

print("\n=== Edited New Code Snippet ===")
print(edited_code)